# Импорт библиотек

In [ ]:
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# Функция создания архитектуры классификатора

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3", name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4", trainable=True, name='BERT')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(69, activation="softmax", name='classifier')(net)
    return tf.keras.Model(text_input, net)

# Инициализация и загрузка весов модели

In [ ]:
model = build_classifier_model()

In [ ]:
model.load_weights('GRNTIClassifier_BERT_v1.h5')

# Конвертация модели

In [ ]:
tf.saved_model.save(model, "model")

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("model")
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)